# Utils

In [ ]:
#| default_exp utils

In [ ]:
#| export
from fastai.vision.all import *
from torchvision.models.feature_extraction import create_feature_extractor

from fastai_datasets.all import *

In [ ]:
#| export
def as_percentage(x, ndigits=2):
    return f'{round(x * 100, ndigits=ndigits)}%'

In [ ]:
#| export
def cut_model_by_name(model, cut):
    graph = create_feature_extractor(model, [cut])
    base_forward = graph.forward
    graph.forward = lambda *args, **kwargs: base_forward(*args, **kwargs)[cut]
    return graph

In [ ]:
classifier = resnet34()
body = cut_model_by_name(classifier, 'avgpool')
test(body, 'avgpool', hasattr)
assert not hasattr(body, 'fc')

In [ ]:
#| export
class MLP(Module):
    def __init__(self, logits: Optional[int], hidden_depth=5, hidden_width=512, features_dim=None):
        super().__init__()
        features_dim = features_dim or hidden_width

        def generate_hidden_layers():
            if hidden_depth == 0:
                return
            if hidden_depth >= 2:
                yield nn.LazyLinear(hidden_width)
                yield nn.ReLU()
                for _ in range(hidden_depth - 2):
                    yield nn.Linear(hidden_width, hidden_width)
                    yield nn.ReLU()
            yield nn.LazyLinear(features_dim)

        self.hidden_layers = nn.Sequential(*generate_hidden_layers())
        self.logits = nn.LazyLinear(logits) if logits else None

    def forward(self, x):
        x = x.flatten(start_dim=1)
        x = self.hidden_layers(x)
        if self.logits:
            x = self.logits(x)
        return x

In [ ]:
#| export
import torch
from torch import nn

from fastai.vision.all import *

class Threshold(nn.Module):
    """Classifies 1D inputs into 2 classes, based on whether they surpass a threshold or not"""
    def __init__(self) -> None:
        super().__init__()
        self.t = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        x = x - self.t
        return torch.stack([x, -x], dim=-1)

In [ ]:
threshold = Threshold()
with torch.no_grad():
    threshold.t[0] = 3

test_eq(threshold(torch.arange(10)).argmax(1), torch.tensor([1]*3+[0]*7))

Since this is such a simple model, we can fit it to data without iterative optimization:

In [ ]:
#| export
@patch
def fit(self: Threshold, x, y):
    """Picks a threshold that maximizes the empirical accuracy"""
    with torch.no_grad():
        def accuracy_for_threshold(t):
            self.t[0] = t
            return accuracy(self(x), y)

        threshold_candidates = np.arange(0.0, 4.0, 0.01)
        self.t[0], accuracy_score = max(((t, accuracy_for_threshold(t)) for t in threshold_candidates),
                                                key=lambda p: p[1])

        return self.t.item(), accuracy_score.item()


In [ ]:
threshold = Threshold()
x = torch.randint(high=10, size=(100,))
chosen_threshold, _ = threshold.fit(x, x < 3)

test_close(chosen_threshold, 3, eps=1)

In [ ]:
#| export
from abc import ABC, abstractmethod
from dataclasses import dataclass
    

@dataclass
class Experiment(ABC):
    """
    Represents some ML experiment, where we which to run a process (e.g. training a model) multiple times (e.g. on different datasets, or to account for stochasticity) and aggregate the results
    """
    model: nn.Module
    data: List[Datasets]
    seed: int = 0
    
    def run(self):
        return ExperimentResults(self._run())

    @return_list
    def _run(self):
        if self.seed is not None:
            set_seed(self.seed, reproducible=True)

        initial_state_dict = deepcopy(self.model.state_dict())

        splits = master_bar(self.data)
        for i, split in enumerate(splits):
            self.dls = split.dls()
            yield self.iteration()
            self.model.load_state_dict(initial_state_dict)

    @abstractmethod
    def iteration(self):
        pass


@dataclass
class ExperimentResults(object):
    """
    Results of an `Experiment`
    """
    stats: List[Any]

    @property
    def collated_stats(self):
        return default_collate(self.stats)

    @property
    def mean_stats(self):
        return self.collated_stats.map(lambda t: t.mean().item())

In [ ]:
class LinearMNIST(Experiment):
    def iteration(self):
        learn = Learner(self.dls, self.model, metrics=accuracy)
        learn.fit(1)
        return learn.validate()
    
res = LinearMNIST(model=nn.Sequential(nn.Flatten(), nn.Linear(3*28*28, 2)),
                  data=[TinyMNIST() for _ in range(3)]).run()

In [ ]:
res.stats

[(#2) [0.13420510292053223,0.9785407781600952],
 (#2) [0.13059759140014648,0.9799714088439941],
 (#2) [0.1380815953016281,0.9799714088439941]]

In [ ]:
res.collated_stats

(#2) [tensor([0.1342, 0.1306, 0.1381], dtype=torch.float64),tensor([0.9785, 0.9800, 0.9800], dtype=torch.float64)]

In [ ]:
res.mean_stats

(#2) [0.1342947632074356,0.9794945319493612]

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()